In [1]:
from sqlalchemy import create_engine, Column, String, Integer, func, event, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry 
from tqdm import tqdm
from shapely.wkt import dumps

import orjson

In [2]:
# Create a base class for our declarative mapping
Base = declarative_base()

# Define your SQLAlchemy model
class GeometryModel(Base):
    __tablename__ = 'geometries'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    geom = Column(Geometry('POLYGON'))

/tmp/ipykernel_90296/2237727833.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
from sqlalchemy_utils import database_exists, create_database
engine = create_engine('postgresql://postgres@localhost:5333/test')#,echo=True)

In [4]:
# Initialize Spatialite extension
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_record):
    with dbapi_connection.cursor() as cursor:
        cursor.execute('CREATE EXTENSION IF NOT EXISTS postgis;')

In [5]:
# Create the table
Base.metadata.create_all(engine)

In [6]:
from tqdm import tqdm

In [7]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select * from geometries")).fetchall()

CPU times: user 5.09 s, sys: 27.7 s, total: 32.8 s
Wall time: 1min 2s


In [8]:
len(results)

1063260

In [11]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,geom from geometries")).fetchall()

CPU times: user 43.4 s, sys: 2.64 s, total: 46.1 s
Wall time: 1min 6s


In [12]:
results[0]

(883, 'te+lumen', '0103000020E6100000010000000503000000000000A0EEE94000000000004CCA4000000000A0EEE94000000000804CCA4000000000A0EEE94000000000004DCA4000000000A0EEE940000 ... (24472 characters truncated) ... FE94000000000004CCA400000000000EFE94000000000004CCA4000000000E0EEE94000000000004CCA4000000000C0EEE94000000000004CCA4000000000A0EEE94000000000004CCA40')

In [13]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,ST_AsGeoJSON(geom) from geometries")).fetchall()

CPU times: user 3.56 s, sys: 1.37 s, total: 4.92 s
Wall time: 23.8 s


In [14]:
results[0]

(883, 'te+lumen', '{"type":"Polygon","coordinates":[[[53109,13464],[53109,13465],[53109,13466],[53109,13467],[53109,13468],[53108,13469],[53107,13470],[53106,13471],[53 ... (10560 characters truncated) ... 13464],[53118,13464],[53117,13464],[53116,13464],[53115,13464],[53114,13464],[53113,13464],[53112,13464],[53111,13464],[53110,13464],[53109,13464]]]}')

In [ ]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,ST_AsEWKB(geom) from geometries")).fetchall()

In [ ]:
results[0]

In [ ]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,ST_AsEWKT(geom) from geometries")).fetchall()

In [ ]:
results[0]

In [ ]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,ST_Centroid(geom) from geometries")).fetchall()

In [ ]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,ST_AsGeoJSON(ST_Centroid(geom)) from geometries")).fetchall()

In [ ]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,ST_AsEWKT(ST_Centroid(geom)) from geometries")).fetchall()

In [ ]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,ST_AsEWKB(ST_Centroid(geom)) from geometries")).fetchall()